In [28]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel, pipeline
import torch

In [29]:
# Load embedding  models
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight, fast, good for similarity

# Load  model
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(embedding_model)

model = AutoModel.from_pretrained(embedding_model).to(device)

# Evaluation model
evaluator = pipeline("text2text-generation", model="google/flan-t5-small", tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


Device set to use cpu


In [30]:
# Example job-specific documents (replace with actual docs)
job_docs = [
    "Python scripting, automation, data analysis using Pandas, NumPy, Matplotlib.",
    "Experience with machine learning frameworks like scikit-learn, TensorFlow, Keras.",
    "Object-oriented programming, version control, API interaction."
]


# Questions to ask
questions = [
    "Can you tell me about your experience with Python?",
    "Describe your experience with machine learning.",
    "How do you approach debugging complex software issues?"
]


# Answers provided by the candidate
candidate_answers = [
    """My experience with Python is quite extensive, spanning several years across various domains. I've primarily leveraged Python for its versatility in data analysis, machine learning, and automation.

    In data analysis, I'm proficient with libraries like NumPy, Pandas, and Matplotlib. I've used them to clean, transform, analyze, and visualize complex datasets, extracting meaningful insights to inform decision-making.

    For machine learning, I've worked extensively with scikit-learn, TensorFlow, and Keras. This includes building and deploying models for tasks such as classification, regression, clustering, and natural language processing. I'm comfortable with the entire ML pipeline, from data preprocessing and feature engineering to model training, evaluation, and hyperparameter tuning.

    Beyond data science, I have strong experience in scripting and automation using Python. I've developed scripts for file manipulation, web scraping, API interactions, and automating repetitive tasks, significantly improving efficiency. I'm also familiar with object-oriented programming principles in Python and have experience with version control systems like Git.""",
    
    """I’ve worked on multiple ML projects. One involved customer churn prediction using Random Forest and XGBoost. I handled data cleaning, feature engineering, and model tuning.

    I’ve also built NLP pipelines for sentiment analysis using TF-IDF + Logistic Regression and LSTM. I prefer TensorFlow and scikit-learn for most projects.""",

    """I first reproduce the issue, review logs, and isolate the failing component. Then I use tools like `pdb`, print statements, and logging. If it's async or multi-threaded, I use `threading` and `concurrent.futures` to track flow. I write unit tests to prevent regression."""
]



interview_answer = """My experience with Python is quite extensive, spanning several years across various domains. I've primarily leveraged Python for its versatility in data analysis, machine learning, and automation.

In data analysis, I'm proficient with libraries like NumPy, Pandas, and Matplotlib. I've used them to clean, transform, analyze, and visualize complex datasets, extracting meaningful insights to inform decision-making.

For machine learning, I've worked extensively with scikit-learn, TensorFlow, and Keras. This includes building and deploying models for tasks such as classification, regression, clustering, and natural language processing. I'm comfortable with the entire ML pipeline, from data preprocessing and feature engineering to model training, evaluation, and hyperparameter tuning.

Beyond data science, I have strong experience in scripting and automation using Python. I've developed scripts for file manipulation, web scraping, API interactions, and automating repetitive tasks, significantly improving efficiency. I'm also familiar with object-oriented programming principles in Python and have experience with version control systems like Git.

Overall, I'm confident in my ability to write clean, efficient, and well-documented Python code to solve a wide range of problems."""

In [31]:
# Helper: Mean Pooling for embeddings
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return (token_embeddings * input_mask_expanded).sum(1) / input_mask_expanded.sum(1)

# Get embedding
def get_embedding(text):
    encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        model_output = model(**encoded)
    return mean_pooling(model_output, encoded['attention_mask']).cpu().numpy()[0]

In [32]:
# Generate embeddings for documents

doc_embeddings = [get_embedding(doc) for doc in job_docs]

# doc_embeddings = embedding_model.encode(job_docs)

# # Generate embeddings for questions
# question_embeddings = embedding_model.encode(questions)

# # Generate embedding for the interview answer
# interview_answer_embedding = embedding_model.encode([interview_answer])[0]

# # Calculate cosine similarity between the interview answer and job-specific documents
# similarity_scores = cosine_similarity([interview_answer_embedding], doc_embeddings)[0]


In [33]:
# RAG retrieval
def rag_retrieve(query, top_k=2):
    query_emb = get_embedding(query)
    sims = [cosine_similarity([query_emb], [doc_emb])[0][0] for doc_emb in doc_embeddings]
    top_indices = np.argsort(sims)[-top_k:][::-1]
    return " ".join([job_docs[i] for i in top_indices])

In [34]:

# Evaluate answer using Flan-T5
def evaluate_answer(question, answer,context=None):
    prompt = f"""You are a professional technical interviewer. 
You are given a job context, an interview question, and a candidate's answer.

Job Context: {context}
Question: {question}
Answer: {answer}

Return your comment and a score between 1 to 10."""
    return evaluator(prompt, max_new_tokens=128)[0]['generated_text']

In [35]:
# Loop, perform RAG, evaluate, save
results = []

for question, answer in zip(questions, candidate_answers):
    context = rag_retrieve(question)
    evaluation = evaluate_answer(question, answer, context)
    
    results.append({
        "question": question,
        "answer": answer,
        "context": context,
        "evaluation": evaluation
    })

# Save results to JSON
with open('interview_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Evaluation complete. Results saved to 'interview_results.json'.")

Evaluation complete. Results saved to 'interview_results.json'.
